<a href="https://colab.research.google.com/github/yang-zhikai/MyStudy/blob/master/cooking_GRU_ACT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install ipdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!/opt/bin/nvidia-smi

#处理工具

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
################################################################################
# Project:  Extracting Action Sequences Based on Deep Reinforcement Learning
# Module:   utils
# Author:   Wenfeng Feng 
# Time:     2017.12
################################################################################

import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('agg') # do not require GUI
from ipdb import set_trace


def str2bool(v):
    """
    Transfrom string command/argument to bool
    """
    return v.lower() in ("yes", "true", "t", "1")


def timeit(f):
    """
    Return time cost of function f
    """
    def timed(*args, **kwargs):
        start_time = time.time()
        result = f(*args, **kwargs)
        end_time = time.time()

        print("   [-] %s : %2.5f sec" % (f.__name__, end_time - start_time))
        return result
    return timed


def get_time():
    """
    Get global time as string
    """
    return time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime())


def save_pkl(obj, path):
    """
    Save pickle file
    """
    with open(path, 'wb') as f:
        pickle.dump(obj, f)


def load_pkl(path):
    """
    Load pickle file
    """
    with open(path, 'rb') as f:
        obj = pickle.load(f)
        return obj


def print_args(args, output_file=''):
    """
    Print all arguments of an argparse instance
    """
    print('\n Arguments:')
    for k, v in sorted(args.__dict__.items(), key=lambda x:x[0]):
        print('{}: {}'.format(k, v))
    if output_file:
        output_file.write('\n Arguments:\n')
        for k, v in sorted(args.__dict__.items(), key=lambda x:x[0]):
            output_file.write('{}: {}\n'.format(k, v))
        output_file.write('\n')


def pos_tagging(agent_mode, domain):
    """
    Part-of-speech tagging, using StanfordPOSTagger
    """
    # import ipdb
    from tqdm import tqdm
    from nltk.tag import StanfordPOSTagger
    # NB! you should download the Stanford postagger and set the following two directories
    # e.g. pos_model = 'stanford/postagger/models/english-bidirectional-distsim.tagger'
    # pos_jar = 'stanford/postagger/stanford-postagger.jar'
    pos_model = ''
    pos_jar = ''
    pos_tagger = StanfordPOSTagger(pos_model, pos_jar)
    
    if agent_mode == 'act':
        indata = pickle.load(open('/content/drive/My Drive/domain.pkl' % domain, 'r'))
    else:
        _, __, indata = pickle.load(open('/content/drive/My Drive/EASDRL/data/refined_%s_data.pkl' % domain, 'r'))

    # ipdb.set_trace()
    # pos_data = []
    pos_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_%s_pos.pkl' % (domain, agent_mode))
    try:
        for i in range(118, len(indata)):
            pos_text = []
            for j in range(len(indata[i])):
                print('Text %d/%d Sent %d/%d' % (i+1, len(indata), j+1, len(indata[i])))
                if len(indata[i][j]) == 0:
                    continue
                last_sent = [w.lower() for w in indata[i][j]['last_sent']]
                this_sent = [w.lower() for w in indata[i][j]['this_sent']]
                pos_sent = [pos_tagger.tag(last_sent), pos_tagger.tag(this_sent)]
                pos_text.append(pos_sent)
            pos_data.append(pos_text)
    except:
        print('Error! i=%d, j=%d' % (i, j))
    save_pkl(pos_data, 'data/%s_%s_pos.pkl' % (domain, agent_mode))

def transfer(infile,output_file):
    indata = load_pkl(infile)[-1]
    data = []
    log = {'wrong_last_sent': 0, 'act_reference_1': 0, 'related_act_reference_1': 0,
            'obj_reference_1': 0, 'non-obj_reference_1': 0}
    for i in range(len(indata)):
        words = []  # all words of a text
        sents = []  # all sentences of a text
        word2sent = {}  # transfer from a word index to its sentence index
        text_acts = []  # all actions of a text
        sent_acts = []  # actions of each sentence
        reference_related_acts = False
        for j in range(len(indata[i])):
            # labeling error: empty sentence
            if len(indata[i][j]) == 0:  
                print('%s, len(indata[%d][%d]) == 0'%(self.domain, i, j))
                continue
            last_sent = indata[i][j]['last_sent']
            this_sent = indata[i][j]['this_sent']
            acts = indata[i][j]['acts']
            
            # labeling error: mis-matched sentences
            if j > 0 and len(last_sent) != len(indata[i][j-1]['this_sent']):
                b1 = len(last_sent)
                b2 = len(indata[i][j-1]['this_sent'])
                for k in range(len(acts)):
                    ai = acts[k]['act_idx']
                    new_act_type = acts[k]['act_type']
                    new_act_idx = ai - b1 + b2
                    new_obj_idxs = [[],[]]
                    for l in range(2):
                        for oi in acts[k]['obj_idxs'][l]:
                            if oi == -1:
                                new_obj_idxs[l].append(oi)
                            else:
                                new_obj_idxs[l].append(oi - b1 + b2)
                        assert len(new_obj_idxs[l]) == len(acts[k]['obj_idxs'][l])
                    new_related_acts = []
                    acts[k] = {'act_idx': new_act_idx, 'obj_idxs': new_obj_idxs,
                            'act_type': new_act_type, 'related_acts': new_related_acts}
                last_sent = indata[i][j-1]['this_sent']
                log['wrong_last_sent'] += 1

            sent = last_sent + this_sent
            last_sent_bias = len(last_sent)
            # pronoun resolution, find the source noun of a pronoun
            reference_obj_flag = False  
            tmp_acts = []
            for k in range(len(acts)):
                act_idx = acts[k]['act_idx']
                obj_idxs = acts[k]['obj_idxs']
                tmp_act_idx = act_idx - last_sent_bias
                if tmp_act_idx < 0:
                    log['act_reference_1'] += 1
                
                tmp_obj_idxs = [[],[]]
                for l in range(2):
                    for oi in obj_idxs[l]:
                        if oi == -1:
                            tmp_obj_idxs[l].append(oi)
                        else:
                            tmp_obj_idxs[l].append(oi - last_sent_bias)
                            if oi - last_sent_bias < 0:
                                reference_obj_flag = True
                    assert len(tmp_obj_idxs[l]) == len(obj_idxs[l])
                tmp_act_type = acts[k]['act_type']
                tmp_related_acts = []
                if len(acts[k]['related_acts']) > 0:
                    for idx in acts[k]['related_acts']:
                        tmp_related_acts.append(idx - last_sent_bias)
                        if idx - last_sent_bias < 0:
                            reference_related_acts = True
                            log['related_act_reference_1'] += 1
                    assert len(tmp_related_acts) == len(acts[k]['related_acts'])
                tmp_acts.append({'act_idx': tmp_act_idx, 'obj_idxs': tmp_obj_idxs,
                            'act_type': tmp_act_type, 'related_acts': tmp_related_acts})
            # assert len(tmp_acts) == len(acts)
            # labeling error: wrong word index in the first sentence 
            if j == 0:
                if reference_obj_flag:
                    log['obj_reference_1'] += 1
                    for ii in range(len(words), len(words)+len(last_sent)):
                        word2sent[ii] = len(sents)
                    words.extend(last_sent)
                    sents.append(last_sent)
                    sent_acts.append({})
                else:
                    if len(last_sent) > 0:
                        log['non-obj_reference_1'] += 1
                        last_sent = []
                        last_sent_bias = len(last_sent)
                        sent = last_sent + this_sent
                        acts = tmp_acts

            
            for ii in range(len(words), len(words)+len(this_sent)):
                word2sent[ii] = len(sents)
            all_word_bias = len(words)
            words.extend(this_sent)
            sents.append(this_sent)
            sent_acts.append(acts)
            all_acts_of_cur_sent = update_acts(words, sent, last_sent_bias, all_word_bias, tmp_acts)
            text_acts.extend(all_acts_of_cur_sent)

        # assert len(word2sent) == len(words)
        # assert len(sents) == len(sent_acts)
        data.append({'words': words, 'acts': text_acts, 'sent_acts': sent_acts,
                    'sents': sents, 'word2sent': word2sent})
    upper_bound, lower_bound = compute_context_len(data)
    print('\nupper_bound: {}\tlower_bound: {}\nlog history: {}\n'.format(upper_bound, lower_bound, log))
    save_pkl(data, outfile)



def update_acts(words, sent, last_sent_bias, all_word_bias, tmp_acts):
    """
    Add all actions of the current sentence to text_acts
    """
    # all indices of the words in the current sentences need to add a last_sent_bias
    all_acts_of_cur_sent = []
    for k in range(len(tmp_acts)):
        act_idx = tmp_acts[k]['act_idx']
        obj_idxs = tmp_acts[k]['obj_idxs']
        text_act_idx = act_idx + all_word_bias
        # labeling error: mis-matched word index
        if sent[act_idx + last_sent_bias] != words[act_idx + all_word_bias]:
            print(sent[act_idx + last_sent_bias], words[act_idx + all_word_bias])
        text_obj_idxs = [[],[]]
        for l in range(2):
            for oi in obj_idxs[l]:
                if oi == -1:
                    text_obj_idxs[l].append(-1)
                else:
                    text_obj_idxs[l].append(oi + all_word_bias)
                    if sent[oi + last_sent_bias] != words[oi + all_word_bias]:
                        ipdb.set_trace()
                        print(sent[oi + last_sent_bias], words[oi + all_word_bias])
            # assert len(text_obj_idxs[l]) == len(obj_idxs[l])
        text_act_type = tmp_acts[k]['act_type']
        text_related_acts = []
        if len(tmp_acts[k]['related_acts']) > 0:
            for idx in tmp_acts[k]['related_acts']:
                text_related_acts.append(idx + all_word_bias)
            # assert len(text_related_acts) == len(tmp_acts[k]['related_acts'])
        acts = {'act_idx': text_act_idx, 'obj_idxs': text_obj_idxs,
                'act_type': text_act_type, 'related_acts': text_related_acts}
        all_acts_of_cur_sent.append(acts)
    return all_acts_of_cur_sent



def compute_context_len(data):
    """
    Compute the length of context for action argument extractor
    the upper_bound/lower_bound indicate how far/near between the action name and its arguments
    the difference between them is used to control the context_len
    e.g. context_len = 2 * upper_bound
    """
    upper_bound = 0
    lower_bound = 0
    for d in data:
        for n in range(len(d['acts'])):
            act = d['acts'][n]['act_idx']
            objs = d['acts'][n]['obj_idxs']
            for l in range(2):
                for obj in objs[l]:
                    if obj == -1:
                        continue
                    if obj - act < lower_bound:
                        lower_bound = obj - act
                    if obj - act > upper_bound:
                        upper_bound = obj - act
    return upper_bound, lower_bound



def plot_results(results, domain, filename):
    """
    Plot training results, called by main
    """
    print('\nSave results to %s' % filename)
    fontsize = 20
    if isinstance(results, list):
        plt.figure()
        plt.plot(range(len(results)), results, label='loss')
        plt.title('domain: %s' % domain)
        plt.xlabel('episodes', fontsize=fontsize)
        plt.legend(loc='best', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)  
        plt.yticks(fontsize=fontsize) 
        plt.savefig(filename, format='pdf')
        print('Success\n')

    else:
        plt.figure(figsize=(16, 20)) # , dpi=300
        plt.subplot(311)
        x = range(len(results['rec']))
        plt.plot(x, results['rec'], label='rec')
        plt.plot(x, results['pre'], label='pre')
        plt.plot(x, results['f1'], label='f1')
        plt.title('domain: %s' % domain, fontsize=fontsize)
        plt.xlabel('episodes', fontsize=fontsize)
        plt.legend(loc='best', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)  
        plt.yticks(fontsize=fontsize) 

        plt.subplot(312)
        plt.plot(range(len(results['rw'])), results['rw'], label='reward')
        plt.xlabel('episodes', fontsize=fontsize)
        plt.legend(loc='best', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)  
        plt.yticks(fontsize=fontsize) 

        if 'loss' in results:
            plt.subplot(313)
            plt.plot(range(len(results['loss'])), results['loss'], label='loss')
            plt.xlabel('episodes', fontsize=fontsize)
            plt.legend(loc='best', fontsize=fontsize)
            plt.xticks(fontsize=fontsize)  
            plt.yticks(fontsize=fontsize) 
        
        plt.subplots_adjust(wspace=0.5,hspace=0.5)
        plt.savefig(filename, format='pdf')
        print('Success\n')



def ten_fold_split_ind(num_data, fname, k, random=True):
    """
    Split data for 10-fold-cross-validation
    Split randomly or sequentially
    Retutn the indecies of splited data
    """
    print('Getting tenfold indices ...')
    if os.path.exists(fname):
        with open(fname, 'rb') as f:
            print('Loading tenfold indices from %s\n' % fname)
            indices = pickle.load(f)
            return indices
    n = num_data/k
    indices = []

    if random:
        tmp_inds = np.arange(num_data)
        np.random.shuffle(tmp_inds)
        for i in range(k):
            if i == k - 1:
                indices.append(tmp_inds[i*n: ])
            else:
                indices.append(tmp_inds[i*n: (i+1)*n])
    else:
        for i in range(k):
            indices.append(range(i*n, (i+1)*n))

    with open(fname, 'wb') as f:
        pickle.dump(indices, f)
    return indices



def index2data(indices, data):
    """
    Obtain k-fold data according to given indices
    """
    print('Spliting data according to indices ...')
    folds = {'train': [], 'valid': []}
    if type(data) == dict:
        keys = data.keys()
        print('data.keys: {}'.format(keys))
        num_data = len(data[keys[0]])
        for i in range(len(indices)):
            valid_data = {}
            train_data = {}
            for k in keys:
                valid_data[k] = []
                train_data[k] = []
            for ind in range(num_data):
                for k in keys:
                    if ind in indices[i]:
                        valid_data[k].append(data[k][ind])
                    else:
                        train_data[k].append(data[k][ind])
            folds['train'].append(train_data)
            folds['valid'].append(valid_data)
    else:
        num_data = len(data)
        for i in range(len(indices)):
            valid_data = []
            train_data = []
            for ind in range(num_data):
                if ind in indices[i]:
                    valid_data.append(data[ind])
                else:
                    train_data.append(data[ind])
            folds['train'].append(train_data)
            folds['valid'].append(valid_data)

    return folds



# if __name__ == '__main__':
#     import ipdb
#     infile = '/content/drive/My Drive/EASDRL/data/online_test/online_labeled_text.pkl' # is online-labeled data
#     outfile = 'online_labeled_text_data.pkl'
#     transfer(infile, outfile)


#构建数据存储

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
################################################################################
# Project:  Extracting Action Sequences Based on Deep Reinforcement Learning
# Module:   ReplayMemory
# Author:   Wenfeng Feng 
# Time:     2017.12
################################################################################

import ipdb
import pickle
import numpy as np

class ReplayMemory:
    """
    Replay memory
    """
    def __init__(self, args, agent_mode):
        print('Initializing ReplayMemory...')
        self.size = args.replay_size
        if agent_mode == 'act':
            self.word_dim = args.word_dim
            self.num_words = args.num_words
            self.state_dim = 3
        elif agent_mode == 'arg':
            self.word_dim = args.word_dim + args.dis_dim
            self.num_words = args.context_len
            self.state_dim = 4
        

        self.actions = np.zeros(self.size, dtype = np.uint8)
        self.rewards = np.zeros(self.size, dtype = np.float16)
        self.states = np.zeros([self.size, self.num_words, self.state_dim], dtype=np.float16)
        self.terminals = np.zeros(self.size, dtype = np.bool)
        self.priority = args.priority
        self.positive_rate = args.positive_rate
        self.batch_size = args.batch_size
        self.count = 0
        self.current = 0

        if args.load_replay:
            self.load(args.save_replay_name)


    def reset(self):
        """
        Reset the replay memory
        """
        print('Reset the replay memory')
        self.actions *= 0
        self.rewards *= 0.0
        self.states *= 0.0
        self.terminals *= False
        self.count = 0
        self.current = 0

        
    def add(self, action, reward, state, terminal):
        """
        Add action, reward, state, and terminal to replay memory
        """
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.states[self.current] = state
        self.terminals[self.current] = terminal
        self.count = max(self.count, self.current + 1)  
        self.current = (self.current + 1) % self.size


    def getMinibatch(self):
        """
        Get a mini-batch of samples
        Memory must include poststate, prestate and history
        Sampling random indexes or with priority
        """
        prestates = np.zeros([self.batch_size, self.num_words, self.state_dim])
        poststates = np.zeros([self.batch_size, self.num_words, self.state_dim])
        if self.priority:
            pos_amount =  int(self.positive_rate*self.batch_size) 

        indexes = []
        count_pos = 0
        count_neg = 0
        count_circle = 0 
        max_circles = 10 * self.batch_size # max times for choosing positive samples or nagative samples
        while len(indexes) < self.batch_size:
            # find random index 
            while True:
                # sample one index (ignore states wraping over) 
                index = np.random.randint(1, self.count - 1)
                # NB! poststate (last state) can be terminal state!
                if self.terminals[index - 1]:
                    continue
                # use prioritized replay trick
                if self.priority:
                    if count_circle < max_circles:
                        # if num_pos is already enough but current ind is also pos sample, continue
                        if (count_pos >= pos_amount) and (self.rewards[index] > 0):
                            count_circle += 1
                            continue
                        # elif num_nag is already enough but current ind is also nag sample, continue
                        elif (count_neg >= self.batch_size - pos_amount) and (self.rewards[index] < 0): 
                            count_circle += 1
                            continue
                    if self.rewards[index] > 0:
                        count_pos += 1
                    else:
                        count_neg += 1
                break
            
            prestates[len(indexes)] = self.states[index - 1]
            indexes.append(index)

        # copy actions, rewards and terminals with direct slicing
        actions = self.actions[indexes]  
        rewards = self.rewards[indexes]
        terminals = self.terminals[indexes]
        poststates = self.states[indexes]
        return prestates, actions, rewards, poststates, terminals


    def save(self, fname, size):
        """
        Save replay memory
        """
        if size > self.size:
            size = self.size
        databag = {}
        databag['actions'] = self.actions[: size]
        databag['rewards'] = self.rewards[: size]
        databag['states'] = self.states[: size]
        databag['terminals'] = self.terminals[: size]
        with open(fname, 'wb') as f:
            print('Try to save replay memory ...')
            pickle.dump(databag, f)
            print('Replay memory is successfully saved as %s' % fname)


    def load(self, fname):
        """
        Load replay memory
        """
        if not os.path.exists(fname):
            print("%s doesn't exist!" % fname)
            return
        with open(fname, 'rb') as f:
            print('Loading replay memory from %s ...' % fname)
            databag = pickle.load(f)
            size = len(databag['states'])
            self.states[: size] = databag['states']
            self.actions[: size] = databag['actions']
            self.rewards[: size] = databag['rewards']
            self.terminals[: size] = databag['terminals']
            self.count = size
            self.current = size
          

#构建智能体

In [ ]:
import ipdb
import time
import random
import numpy as np
from tqdm import tqdm

class Agent:
    """
    Reinforcement Learning Agent
    """
    def __init__(self, environment, replay_memory, deep_q_network, args):
        print('Initializing the Agent...')
        self.env = environment
        self.mem = replay_memory
        self.net = deep_q_network
        self.agent_mode = args.agent_mode
        self.num_words = args.num_words
        self.batch_size = args.batch_size
        self.num_actions = args.num_actions

        self.exp_rate_start = args.exploration_rate_start
        self.exp_rate_end = args.exploration_rate_end
        self.exp_decay_steps = args.exploration_decay_steps
        self.exploration_rate_test = args.exploration_rate_test
        self.total_train_steps = args.start_epoch * args.train_steps

        self.train_frequency = args.train_frequency
        self.train_repeat = args.train_repeat
        self.target_steps = args.target_steps
        self.random_play = args.random_play
        self.display_training_result = args.display_training_result
        
        self.steps = 0  # use to decrease the reward during time steps 
        self.filter_act_ind = args.filter_act_ind 

    
    def _restart(self, train_flag, init=False):
        """
        Restart the environment and self.steps (time step)
        where self.steps indicate the index of the current word
        """
        self.steps = 0
        self.env.restart(train_flag, init)


    def _explorationRate(self):
        """
        Calculate decaying exploration rate
        """
        if self.total_train_steps < self.exp_decay_steps:
            return self.exp_rate_start - self.total_train_steps * \
            (self.exp_rate_start - self.exp_rate_end) / self.exp_decay_steps
        else:
            return self.exp_rate_end
 

    def step(self, exploration_rate):
        """
        Go a step forward
        """
        # exploration rate determines the probability of random moves
        if random.random() < exploration_rate:
            action = np.random.randint(self.num_actions)
        else:
            # otherwise choose action with highest Q-value
            current_state = self.env.getState()
            qvalues = self.net.predict(current_state)
            action = np.argmax(qvalues[0])
           
            
            
        # perform the action  
        reward = self.env.act(action, self.steps)
        state = self.env.getState()
        terminal = self.env.isTerminal()
        
        results = []
        self.steps += 1
        # at a terminal time step, compute the scores of training results
        if terminal:
            results = self.compute_f1(self.display_training_result)
            self.steps = 0
            # give a bonus to the terminal actions, it could be further fine-tuned 
            reward += 2   

        return action, reward, state, terminal, results

    def train(self, train_steps, train_episodes, restart_init):
        '''
        Perform a number of steps for training
        '''
        #ipdb.set_trace()
        print('\n\n Training... ')
        trained_texts = 0
        ep_loss, ep_rewards = [], []
        ep_results = {'rec': [], 'pre': [], 'f1': [], 'loss': [], 'rw': []}
        if restart_init:
            self._restart(train_flag=True, init=True)
        for i in range(train_steps):
            if self.random_play:
                action, reward, state, terminal, results = self.step(1)
               
                
            else:
                action, reward, state, terminal, results = self.step(self._explorationRate())
                
                # set_trace()
                self.mem.add(action, reward, state, terminal)
                

                # Update target network every target_steps steps
                if self.target_steps and i % self.target_steps == 0:
                    self.net.update_target_network()

                # train after every train_frequency steps
                if self.mem.count > self.mem.batch_size and i % self.train_frequency == 0:
                    # train for train_repeat times
                    for j in range(self.train_repeat):
                        # sample minibatch
                        minibatch = self.mem.getMinibatch()
                       
                        prestates, actions, rewards, poststates, terminals=minibatch
                        
                      
                        # train the network
                        loss = self.net.train(minibatch)
                        
                        ep_loss.append(loss)
                      
            # increase number of training steps for epsilon decay
            ep_rewards.append(reward)
            self.total_train_steps += 1
            if terminal:
                if len(ep_loss) > 0:
                    avg_loss = sum(ep_loss) / len(ep_loss)
                    max_loss = max(ep_loss)
                    min_loss = min(ep_loss)
                    print('max_loss: {:>6.6f}\t min_loss: {:>6.6f}\t avg_loss: {:>6.6f}'.format(
                          max_loss, min_loss, avg_loss))
                    ep_results['loss'].append(avg_loss)
                
                ep_results['rec'].append(results[-3])
                ep_results['pre'].append(results[-2])
                ep_results['f1'].append(results[-1])
                ep_results['rw'].append(sum(ep_rewards)/len(ep_rewards))
                ep_loss, ep_rewards = [], []
                trained_texts += 1
                self._restart(train_flag=True)
                
            if self.env.train_epoch_end_flag or trained_texts >= train_episodes:
                print("loss list{}".format(ep_results['loss']))
                break

        return ep_results
    

    def test(self, test_steps, outfile):
        '''
        Perform many steps until all test texts are used
        '''
        print('\n\n Testing ...')
        t_f1 = 0
        t_rec = 0
        t_pre = 0
        t_total_acts = 0
        t_right_acts = 0
        t_tagged_acts = 0
        cumulative_reward = 0
        self._restart(train_flag=False, init=True)
        for test_step in range(test_steps):
            if self.random_play:
                a, r, s, t, rs = self.step(1)
            else:
                a, r, s, t, rs = self.step(self.exploration_rate_test)
            cumulative_reward += r
            if t:
                t_total_acts += rs[0] 
                t_right_acts += rs[1]
                t_tagged_acts += rs[2]
                self._restart(train_flag=False)    
                
            if self.env.valid_epoch_end_flag:
                break   

        average_reward = cumulative_reward / (test_step + 1)
        results = {'rec': [], 'pre': [], 'f1': []}
        self.basic_f1(t_total_acts, t_right_acts, t_tagged_acts, results)
        t_rec = results['rec'][-1]
        t_pre = results['pre'][-1]
        t_f1 = results['f1'][-1]

        outfile.write('\n\nSummary:\n')
        outfile.write('total_act: %d\t right_act: %d\t tagged_act: %d\n' % (t_total_acts, t_right_acts, t_tagged_acts))  
        outfile.write('rec: %f\t pre: %f\t f1: %f\n' % (t_rec, t_pre, t_f1))
        outfile.write('\ncumulative reward: %f\t average reward: %f\n' % (cumulative_reward, average_reward))
        print('\n\nSummary:')
        print('total_act: %d\t right_act: %d\t tagged_act: %d' % (t_total_acts, t_right_acts, t_tagged_acts))  
        print('rec: %f\t pre: %f\t f1: %f' % (t_rec, t_pre, t_f1))
        print('\ncumulative reward: %f\t average reward: %f\n' % (cumulative_reward, average_reward))
        return t_rec, t_pre, t_f1, average_reward


    def compute_f1(self, display):
        """
        Compute f1 score for the current text
        """
        text_vec_tags = self.env.text_vec[:,-1]  # ground truth
        state_tags = self.env.state[:,-1]
        # print("stage_tags:{}".format(state_tags))
        # print("text_vec_tags:{}".format(text_vec_tags))
        
        # action names are not action arguments, so mask them when extracting action arguments
        if self.agent_mode == 'arg' and self.filter_act_ind: 
            state_tags[self.env.current_text['act_inds']] = 1
        
        total_words = self.num_words
        temp_words = len(self.env.current_text['tokens'])
        if temp_words > total_words:
            temp_words = total_words

        record_ecs_act_inds = []  # record indices of exclusive action names or arguments  
        right_acts = tagged_acts = total_acts = 0
        # go through all words of the current text
        for i in range(temp_words):
            if state_tags[i] == 2:      # tagged as an action (name/argument)
                tagged_acts += 1
            if text_vec_tags[i] == 2:   # essential action (name/argument)
                total_acts += 1
                if state_tags[i] == 2:  # extract
                    right_acts += 1
            elif text_vec_tags[i] == 3: # optional action (name/argument)
                if state_tags[i] == 2:  # extract
                    total_acts += 1
                    right_acts += 1
            elif text_vec_tags[i] == 4: # exclusive action (name/argument)
                if i not in record_ecs_act_inds:
                    total_acts += 1
                    record_ecs_act_inds.append(i)
                # for action argument extractor
                if self.agent_mode == 'arg':
                    right_flag = True
                    if i in self.env.current_text['obj_inds'][0]:
                        exc_objs = self.env.current_text['obj_inds'][1]
                    else:
                        exc_objs = self.env.current_text['obj_inds'][0]
                    record_ecs_act_inds.extend(exc_objs)
                    # if select an action argument which is exclusive with the current one
                    # then the operation is wrong
                    for oi in exc_objs:
                        if state_tags[oi] == 2:
                            right_flag = False
                            break
                    if state_tags[i] == 2 and right_flag:
                        right_acts += 1
                # for action name extractor
                else:
                    assert i in self.env.current_text['act2related']
                    exclusive_act_inds = self.env.current_text['act2related'][i]
                    record_ecs_act_inds.extend(exclusive_act_inds)
                    # if select an action name which is exclusive with the current one
                    # then exclusive_flag is True, the operation is wrong
                    exclusive_flag = False
                    for ind in exclusive_act_inds:
                        if state_tags[ind] == 2: # extracted as an action
                            exclusive_flag = True
                            break
                    if not exclusive_flag and state_tags[i] == 2: # extract
                        right_acts += 1

        results = {'rec': [], 'pre': [], 'f1': []}
        self.basic_f1(total_acts, right_acts, tagged_acts, results)
        rec = results['rec'][-1]
        pre = results['pre'][-1]
        f1 = results['f1'][-1]
        if display:
            print('rec: {:>13.6f}\t pre: {:>13.6f}\t f1: {:>14.6f}'.format(rec, pre, f1))
        return total_acts, right_acts, tagged_acts, rec, pre, f1


    def basic_f1(self, total, right, tagged, results):
        """
        Compute f1 scores
        """
        rec = pre = f1 = 0.0
        if total > 0:
            rec = right / float(total)
        if tagged > 0:
            pre = right / float(tagged)
        if rec + pre > 0:
            f1 = 2 * pre * rec / (pre + rec)
        results['rec'].append(rec)
        results['pre'].append(pre)
        results['f1'].append(f1)

#构建环境

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
################################################################################
# Project:  Extracting Action Sequences Based on Deep Reinforcement Learning
# Module:   Environment
# Author:   Wenfeng Feng 
# Time:     2017.12
################################################################################

import re
import ipdb
import time
import pickle
import numpy as np
from copy import deepcopy



class Environment:
    """
    Environment
    """
    def __init__(self, args, agent_mode):
        print('Initializing the Environment...')  
        self.agent_mode = agent_mode 
        self.domain = args.domain
        self.dis_dim = args.dis_dim
        self.tag_dim = args.tag_dim
        self.word_dim = args.word_dim
        self.num_words = args.num_words
        self.context_len = args.context_len
        self.action_rate = args.action_rate
        self.use_act_rate = args.use_act_rate
        self.use_act_att = args.use_act_att
        self.reward_base = args.reward_base
        self.ra = args.reward_assign
        self.word2ids = args.word2ids
       
        self.fold_id = args.fold_id
        self.k_fold = args.k_fold
        self.k_fold_indices = args.k_fold_indices

        self.terminal_flag = False
        self.train_epoch_end_flag = False
        self.valid_epoch_end_flag = False
        self.train_text_idx = -1
        self.valid_text_idx = -1
        self.build_dict()
        self.get_fold_data(args.fold_id)
        
        args.num_pos = len(self.pos_dict) + 1
        temp_size = self.train_steps * args.epochs + self.valid_steps
        if temp_size < args.replay_size:
            args.replay_size = temp_size
        if args.train_episodes == 0:
            args.train_episodes = self.num_train
        args.valid_episodes = self.num_valid
        args.train_steps = self.train_steps
        args.valid_steps = self.valid_steps
        
        
    def build_dict(self):
        """
        Buiild part-of-speech dictionary
        """
        self.pos_dict = {'PAD': 0}
        for domain in ['cooking', 'win2k', 'wikihow']:
            sent_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_dependency.pkl' % domain)
            for sents in sent_data:
                for sent in sents:
                    for word, pos in sent:
                        if pos not in self.pos_dict:
                            self.pos_dict[pos] = len(self.pos_dict)

        print('len(pos_dict): %d' % len(self.pos_dict))


    def get_fold_data(self, fold_id):
        """
        Get a fold of data (train & validate)
        """
        if self.agent_mode == 'act':
            data = self.read_act_texts()
        else:
            data = self.read_arg_sents()
        print('\n\nGet new fold data')
        self.train_data = data['train'][fold_id]
        self.valid_data = data['valid'][fold_id]
        self.train_steps = len(self.train_data) * self.num_words
        self.valid_steps = len(self.valid_data) * self.num_words

        self.num_train = len(self.train_data)
        self.num_valid = len(self.valid_data)
        print('training texts: %d\tvalidation texts: %d' % (len(self.train_data), len(self.valid_data)))
        print('self.train_steps: %d\tself.valid_steps: %d\n\n' % (self.train_steps, self.valid_steps))
       


    def read_act_texts(self):
        """
        Read data for action name extractor
        PS: pos_data is generated by the function ``pos_tagging" in utils.py
        """
        text_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_labeled_text_data.pkl' % self.domain)
        pos_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_dependency.pkl' % self.domain)
        act_texts = []
      
        #ipdb.set_trace()
        for i in range(len(text_data)):
            act_text = {}
            act_text['tokens'] = text_data[i]['words']
            act_text['sents'] = text_data[i]['sents']
            act_text['acts'] = text_data[i]['acts']
            act_text['sent_acts'] = text_data[i]['sent_acts']
            act_text['word2sent'] = text_data[i]['word2sent']
            act_text['tags'] = np.ones(shape=(len(text_data[i]['words'])), dtype=np.int32)
            act_text['act2related'] = {}
            for acts in text_data[i]['acts']:
                act_text['act2related'][acts['act_idx']] = acts['related_acts']
                act_text['tags'][acts['act_idx']] = acts['act_type'] + 1 # 2, 3, 4
            act_text['pos']=[]
            for sent in pos_data[i]:
                for word, pos in sent:
                    act_text['pos'].append(self.pos_dict[pos])
                 
            self.create_matrix(act_text)
            act_texts.append(act_text)
        act_indices = ten_fold_split_ind(len(act_texts), self.k_fold_indices, self.k_fold)
        act_data = index2data(act_indices, act_texts)
        return act_data


    def read_arg_sents(self):
        """
        Read data for action argument extractor
        PS: pos_data is generated by the function ``pos_tagging" in utils.py
        """
        
        indata = load_pkl('/content/drive/My Drive/EASDRL/data/refined_%s_data.pkl' % self.domain)[-1]
        pos_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_arg_pos.pkl' % self.domain)
        
        arg_sents = []
        
        # ipdb.set_trace()
        for i in range(len(indata)):
            for j in range(len(indata[i])):
                if len(indata[i][j]) == 0:
                    continue
                # -1 obj_ind refer to UNK
                words = indata[i][j]['last_sent'] + indata[i][j]['this_sent']+['unknown'] 
                pos = [self.pos_dict[p] for w, p in pos_data[i][j][0] + pos_data[i][j][1]] + [0]
                if len(words) != len(words):
                    ipdb.set_trace()
                    print('len(words) != len(words)')
                sent_len = len(words)
                act_inds = [a['act_idx'] for a in indata[i][j]['acts'] if a['act_idx'] < self.num_words]
                for k in range(len(indata[i][j]['acts'])):
                    act_ind = indata[i][j]['acts'][k]['act_idx']
                    obj_inds = indata[i][j]['acts'][k]['obj_idxs']
                    arg_sent = {}
                    arg_tags = np.ones(sent_len, dtype=np.int32)
                    if len(obj_inds[1]) == 0:
                        arg_tags[obj_inds[0]] = 2 # essential objects
                    else:
                        arg_tags[obj_inds[0]] = 4 # exclusive objects
                        arg_tags[obj_inds[1]] = 4 # exclusive objects
                    # generate distance representation
                    position = np.zeros(sent_len, dtype=np.int32)
                    position.fill(act_ind)
                    distance = np.abs(np.arange(sent_len) - position)
                    
                    arg_sent['tokens'] = words
                    arg_sent['tags'] = arg_tags
                    arg_sent['pos'] = deepcopy(pos)
                    arg_sent['act_ind'] = act_ind
                    arg_sent['distance'] = distance
                    arg_sent['act_inds'] = act_inds
                    arg_sent['obj_inds'] = obj_inds
                    self.create_matrix(arg_sent)
                    arg_sents.append(arg_sent)
        # get k-fold split data
        arg_indices = ten_fold_split_ind(len(arg_sents), self.k_fold_indices, self.k_fold)
        arg_data = index2data(arg_indices, arg_sents)
       
        return arg_data
        

    def create_matrix(self, sentence):
        """
        Create state representation
        """
        # get word vectors from pre-trained word2vec model
       
        
        sent_ids=[]
        for word in sentence["tokens"]:
            if word in list(self.word2ids.keys()):
                sent_ids.append(self.word2ids[word])
            elif word.lower() in list(self.word2ids.keys()):
                sent_ids.append(self.word2ids[word.lower()])
            else:
                sent_ids.append(0)
        
        # padding
        
        
        pad_len = self.num_words - len(sentence["tokens"])
       
        if self.agent_mode == 'act':
            if pad_len > 0:
                sent_ids.extend([0]*pad_len)
                sentence['tags'] = np.concatenate((np.array(sentence['tags']), np.ones(pad_len ,dtype=np.int32)))
                sentence['pos'].extend([0]*pad_len)
               
                
                
            else:
                sent_ids = sent_ids[:self.num_words]
                sentence['pos'] = sentence['pos'][: self.num_words]
                sentence['tokens'] = sentence['tokens'][: self.num_words]
                sentence['tags'] = np.array(sentence['tags'])[: self.num_words]
           # for action names extraction
            sentence['sent_vec'] = np.array(sent_ids)[:,np.newaxis]
        else: # self.agent_mode == 'arg':
            distance = np.zeros([self.num_words, 1])
            act_vec = sent_ids[sentence['act_ind']]  # word vector of the input action 
            # compute dot attention between the input action and its context 
            # attention = np.sum(sent_vec * act_vec, axis=1)  
            # attention = np.exp(attention)
            # attention /= sum(attention)
            if pad_len > 0:
                sent_vec = np.concatenate((np.array(sent_ids)[:,np.newaxis], np.zeros([pad_len, 1])))
                sentence['tags'] = np.concatenate((np.array(sentence['tags']), np.ones(pad_len,dtype=np.int32)))
                sentence['pos'].extend([0]* pad_len)
                # attention = np.concatenate((attention, np.zeros(pad_len)))
                for d in range(len(sentence['distance'])):
                    distance[d] = sentence['distance'][d]
            else:
                sent_vec = np.array(sent_ids[: self.num_words])[:,np.newaxis]
                sentence['tokens'] = sentence['tokens'][: self.num_words]
                sentence['tags'] = np.array(sentence['tags'])[: self.num_words]
                sentence['pos'] = sentence['pos'][: self.num_words]
                # attention = attention[: self.num_words]
                for d in range(self.num_words):
                    distance[d] = sentence['distance'][d]
            # ipdb.set_trace()
            # if self.use_act_att: # apply attention to word embedding
            #     sent_vec = attention.reshape(-1, 1) * sent_vec
            sentence['sent_vec'] = np.concatenate((sent_vec, distance), axis=1)
            
  

       
        sentence['pos'] = np.array(sentence['pos'])[:, np.newaxis]
        sentence['tags'].shape = (self.num_words,1)
        

    def init_predict_act_text(self, raw_text, isKeras=True):
        """
        Generate action name state representation of an online text input
        """
        # raw_text = re.sub(r'\n|\r|\(|\)|,|;', ' ', raw_text)
        # raw_text = re.split(r'\. |\? |\! ', raw_text)
        tag_dim=2
        text = {'tokens': [], 'sents': [], 'word2sent': {}}
        for s in raw_text:
            words = s.split()
            if len(words) > 0:
                for i in range(len(words)):
                    text['word2sent'][i + len(text['tokens'])] = [len(text['sents']), i]
                text['tokens'].extend(words)
                text['sents'].append(words)

        sent_vec = np.zeros([self.num_words, self.word_dim + tag_dim])
        for i, w in enumerate(text['tokens']):
            if i >= self.num_words:
                break
            if w in self.word2vec.vocab:
                sent_vec[i][: self.word_dim] = self.word2vec[w]

        self.state = sent_vec
        self.terminal_flag = False
        self.current_text = text


    def init_predict_arg_text(self, act_idx, text, isKeras=True):
        """
        Generate action argument state representation of an online text input
        """
        tag_dim = 2
        self.terminal_flag = False
        sents = text['sents']
        word2sent = text['word2sent']
        sent_idx = word2sent[act_idx][0]
        word_ids = []
        this_sent = sents[sent_idx]
        if sent_idx > 0: # use the former sentence and current one
            last_sent = sents[sent_idx - 1]
            for k, v in word2sent.items():
                if v[0] == sent_idx or v[0] == sent_idx - 1:
                    word_ids.append(k)
        else:
            last_sent = []
            for k, v in word2sent.items():
                if v[0] == sent_idx:
                    word_ids.append(k)
        words = last_sent + this_sent + ['UNK']
        end_idx = max(word_ids) # the last index of words of these two sents
        start_idx = min(word_ids)
        sent_len = len(words)

        position = np.zeros(sent_len, dtype=np.int32)
        position.fill(act_idx - start_idx)
        distance = np.abs(np.arange(sent_len) - position)
        sent_vec = np.zeros([self.context_len, self.word_dim + self.dis_dim + 2])
        for i, w in enumerate(words):
            if i >= self.context_len:
                break
            if w in self.word2vec.vocab:
                sent_vec[i][: self.word_dim] = self.word2vec[w]
            sent_vec[i][self.word_dim: self.word_dim + self.dis_dim] = distance[i]
        self.state = sent_vec
        self.current_text = {'tokens': words, 'word2sent': word2sent, 'distance': distance}
        return last_sent, this_sent


    def act_online(self, action, word_ind):
        """
        Perform an action of the RL Agent, for online use
        """
        #self.state[word_ind, -1] = action + 1
        self.state[word_ind, -self.tag_dim:] = action + 1
        if word_ind + 1 >= len(self.current_text['tokens']):
            self.terminal_flag = True


    def restart(self, train_flag, init=False):
        """
        Start a new text and get its state representation
        """
        if train_flag:
            if init:
                self.train_text_ind = -1
                self.train_epoch_end_flag = False
            self.train_text_ind += 1
            if self.train_text_ind >= len(self.train_data):
                self.train_epoch_end_flag = True
                print('\n\n-----train_epoch_end_flag = True-----\n\n')
                return
            self.current_text = self.train_data[self.train_text_ind%self.num_train]
            print('\ntrain_text_ind: %d of %d' % (self.train_text_ind, len(self.train_data)))
        else:
            if init:
                self.valid_text_ind = -1
                self.valid_epoch_end_flag = False
            self.valid_text_ind += 1
            if self.valid_text_ind >= len(self.valid_data):
                self.valid_epoch_end_flag = True
                print('\n\n-----valid_epoch_end_flag = True-----\n\n')
                return
            self.current_text = self.valid_data[self.valid_text_ind]
            print('\nvalid_text_ind: %d of %d' % (self.valid_text_ind, len(self.valid_data)))
        
        
        
        self.text_vec = np.concatenate((self.current_text['sent_vec'], 
                                        self.current_text['pos'],
                                        self.current_text['tags']), 
                                        axis=1)
        self.state = self.text_vec.copy() 
        self.state[:, -1] = 0   # initialize all operation to NULL
        self.terminal_flag = False

        
    def act(self, action, word_ind):
        '''
        Performs action and returns reward
        even num refers to tagging action, odd num refer to non-action

        '''
        
        self.state[word_ind, -1] = action +1
        # amount of tagged actions 
        t_a_count = sum(self.state[: word_ind + 1, -1]) - (word_ind + 1)
        t_a_rate = float(t_a_count) / self.num_words

        label = self.text_vec[word_ind,-1]
        if self.agent_mode == 'arg':
            # text_vec is labelled data
            if label == 2:
                if action == 1:
                    reward = self.ra[1] * self.reward_base
                else:
                    reward = -self.ra[1] * self.reward_base
            elif label == 4:
                right_flag = True
                if word_ind in self.current_text['obj_inds'][0]:
                    exc_objs = self.current_text['obj_inds'][1]
                else:
                    exc_objs = self.current_text['obj_inds'][0]
                for oi in exc_objs: # exclusive objs
                    if self.state[oi, -1] == 2:
                        right_flag = False
                        break
                if action == 1 and right_flag:
                    reward = self.ra[2] * self.reward_base
                elif action == 2 and not right_flag:
                    reward = self.ra[2] * self.reward_base
                elif action == 2 and word_ind != self.current_text['obj_inds'][1][-1]:
                    reward = self.ra[2] * self.reward_base
                else:
                    reward = -self.ra[2] * self.reward_base
            else: #if label == 1: # non_action 
                if action == 0:
                    reward = self.ra[0] * self.reward_base
                else:
                    reward = -self.ra[0] * self.reward_base

        else: # self.agent_mode == 'act'
            if label == 2: #required action
                if action == 1: # extracted as action
                    reward = self.ra[1] * self.reward_base
                else: # filtered out
                    reward = -self.ra[1] * self.reward_base
            elif label == 3: #optional action
                if action == 1:
                    reward = self.ra[0] * self.reward_base
                else:
                    reward = 0.0
            elif label == 4: # exclusive action
                #ipdb.set_trace()
                assert word_ind in self.current_text['act2related']
                exclusive_act_inds = self.current_text['act2related'][word_ind]
                exclusive_flag = False
                not_biggest_flag = False
                for ind in exclusive_act_inds:
                    if self.state[ind, -1] == 2: # extracted as action
                        exclusive_flag = True
                    if ind > word_ind:
                        not_biggest_flag = True
                if action == 1 and not exclusive_flag:
                # extract current word and no former exclusive action was extracted
                    reward = self.ra[2] * self.reward_base
                elif action == 0 and exclusive_flag:
                # filtered out current word because one former exclusive action was extracted
                    reward = self.ra[2] * self.reward_base
                elif action == 0 and not_biggest_flag:
                # filtered out current word and at least one exclusive action left 
                    reward = self.ra[2] * self.reward_base
                else:
                    reward = -self.ra[2] * self.reward_base
            else: #if label == 1: # non_action 
                if action == 0:
                    reward = self.ra[0] * self.reward_base
                else:
                    reward = -self.ra[0] * self.reward_base
        
        if self.use_act_rate and reward != 0:
            if t_a_rate <= self.action_rate and reward > 0:
                reward += 5.0 * np.square(t_a_rate) * self.reward_base
            else:
                reward -= 5.0 * np.square(t_a_rate) * self.reward_base
        # all words of current text are tagged, break
        if word_ind + 1 >= len(self.current_text['tokens']):
            self.terminal_flag = True
        
        return reward


    def getState(self):
        '''
        Gets current text state
        '''
        return self.state


    def isTerminal(self):
        '''
        Returns if tag_actions is done
        if all the words of a text have been tagged, then terminate
        '''
        return self.terminal_flag


#注意力机制


##Feed Forward注意力机制

In [ ]:
import  tensorflow.keras.initializers

import tensorflow as tf


class Attention(tf.keras.layers.Layer):
    def __init__(self,
                 W_regularizer=None,
                 b_regularizer=None,
                 W_constraint=None,
                 b_constraint=None,
                 bias=True,
                 **kwargs
                 ):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        super(Attention, self).__init__()
        self.bias = bias
        self.init = tf.keras.initializers.get('glorot_uniform')

    def build(self, input_shape):
        '''
        :param input_shape:
        :return:
        '''
        self.output_dim = input_shape[-1]
        self.W = self.add_weight(
                                 name='{}_W'.format(self.name),
                                 shape=(input_shape[2], 1),
                                 initializer=self.init,
                                 trainable=True
                                 )
        if self.bias:
            self.b = self.add_weight(
                                     name='{}_b'.format(self.name),
                                     shape=(input_shape[1], 1),
                                     initializer='zero',
                                     trainable=True
                                     )
        else:
            self.b = None

        self.built = True

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs, mask=None):
        # (N, step, d), (d, 1)  ==>   (N, step, 1)
        e = tf.matmul(inputs, self.W, )
        if self.bias:
            e += self.b
        e = tf.tanh(e)
        a = tf.nn.softmax(e, axis=1)
        # (N, step, d) (N, step, 1) ====> (N, step, d)
        c = inputs*a
        # (N, d)
        c = tf.reduce_sum(c, axis=1)
        return c

    def get_config(self):
        return {'units': self.output_dim}


##自注意力机制

In [ ]:
import tensorflow as tf
from tensorflow import keras
import math
class self_attention(keras.layers.Layer):
  def __init__(self,hidden_dim):
    super(self_attention,self).__init__()
    
    self.hidden_dim = hidden_dim
    self.norm = keras.layers.BatchNormalization()
  def build(self,input_shape):
    #输入维度为(samples,steps,features)
    self.Wq=self.add_weight(name="Wq",shape=(input_shape[-1],self.hidden_dim),
                            initializer="glorot_uniform",
                            trainable=True)
    self.Wk=self.add_weight(name="Wk",shape=(input_shape[-1],self.hidden_dim),
                            initializer="glorot_uniform",
                            trainable=True)
    self.Wv=self.add_weight(name="Wv",shape=(input_shape[-1],self.hidden_dim),
                            initializer="glorot_uniform",
                            trainable=True)
    self.built=True
  def call(self,inputs):
    
    Q=tf.matmul(inputs,self.Wq)
    K=tf.matmul(inputs,self.Wk)
    V=tf.matmul(inputs,self.Wv)

    #转制进行矩阵乘法
    e=tf.matmul(Q,tf.transpose(K,perm=[0,2,1]))
    #归一化处理
    e=tf.divide(e,math.sqrt(self.hidden_dim))
    #在维度1上做softmax得到注意力权重
    e=tf.nn.softmax(e,axis=1)
    #输入向量乘以注意力权重
    e=tf.matmul(e,V)
    #在维度1上做加法得到最后的输出
    out=tf.reduce_sum(e,axis=1)
   
    
    return out


#使用tf2.0创建DQN

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
################################################################################
# Project:  Extracting Action Sequences Based on Deep Reinforcement Learning
# Module:   KerasEADQN
# Author:   Wenfeng Feng 
# Time:     2017.12
################################################################################

import ipdb 

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D,LSTM,Activation,Flatten,Dense,Input,concatenate,BatchNormalization,GRU,SimpleRNN
from tensorflow.keras import Model



        
pos_dict = {'PAD': 0}
for domain in ['cooking', 'win2k', 'wikihow']:
    sent_data = load_pkl('/content/drive/My Drive/EASDRL/data/%s_dependency.pkl' % domain)
    for sents in sent_data:
        for sent in sents:
            for word, pos in sent:
                if pos not in pos_dict:
                    pos_dict[pos] = len(pos_dict)

pos_value=[]
for key,value in pos_dict.items():
    pos_value.append(value)
   
tag_value=[]
for i in range(3):
    tag_value.append(i)




#自定义嵌入层
class Embedding_layer(keras.layers.Layer):
    def __init__(self,emb_matrix):
            
        super(Embedding_layer,self).__init__()
        # tag_embedding
        self.tag_emb_layer = tf.keras.layers.Embedding(trainable=True,input_dim=3,output_dim=50,dtype=tf.float32)
        #pos_embedding
        self.pos_emb_layer = tf.keras.layers.Embedding(trainable=True,input_dim=37,output_dim=50 ,dtype=tf.float32)
        #distance embedding for action arguments extraction
        self.dis_emb_layer = tf.keras.layers.Embedding(trainable=True,input_dim=200,output_dim=50,dtype=tf.float32)
        # word_embedding
        self.word_emb_layer = tf.keras.layers.Embedding(input_dim=len(emb_matrix),output_dim=50,mask_zero=True,
                                 weights=[emb_matrix],trainable=True)
    def call(self,state):
        # for action names extraction
        word_emb = int(state[:,:,0])
        pos_ind = int(state[:,:,1])
        tag_ind = int(state[:,:,2])


        # for action arguments extraction
        # word_emb = int(state[:,:,0])
        # dis      = int(state[:,:,1]) 
        # pos_ind = int(state[:,:,2])
        # tag_ind = int(state[:,:,3])
        
        
        # dis_emb = self.dis_emb_layer(dis) 
        pos_emb = self.pos_emb_layer(pos_ind)
        tag_emb = self.tag_emb_layer(tag_ind)
        word_emb = self.word_emb_layer(word_emb)
        # # for action arguments extraction
        # out=tf.keras.layers.concatenate([word_emb,dis_emb,pos_emb,tag_emb],axis=2)
        
        #for action names extraction
        out=tf.keras.layers.concatenate([word_emb,pos_emb,tag_emb],axis=2)
        return out
            





class DeepQLearner:
    """
    Deep Q-Network, in keras
    """
    def __init__(self, args, agent_mode):
        print('Initializing the DQN...')
        self.word_dim = args.word_dim
        self.tag_dim = args.tag_dim
        self.num_pos = args.num_pos
        self.pos_dim = args.pos_dim
        self.dropout = args.dropout
        self.optimizer = args.optimizer
        self.dense_dim = args.dense_dim
        self.batch_size = args.batch_size
        self.gamma = args.gamma
        self.learning_rate = args.learning_rate
        self.num_actions = args.num_actions
        self.num_filters = args.num_filters
        self.emb_matrix = args.emb_matrix
        if agent_mode == 'act':
            self.num_words = args.num_words
            self.input_dim = 3
            self.units = 256
        elif agent_mode == 'arg':
            self.num_words = args.context_len
            self.input_dim = 4
            self.emb_dim = args.word_dim + args.dis_dim + 100
            self.units = 256
        self.emb = Embedding_layer(self.emb_matrix)
        self.attention=self_attention(256)
            
        self.target_model=self.build_dqn()
        self.model=self.build_dqn()
        self.compile_model()
    
   
    
   
            
    def build_dqn(self):
        """
        Build Text-CNN
        """
        
        # ipdb.set_trace()
        units = self.units #filter width
        fn = self.num_filters  #filter num
        # inputs = Input(shape=(self.num_words, self.emb_dim, 1))
        x = Input(shape=(self.num_words, self.input_dim))
        inputs=self.emb(x)
        
        out_forward = GRU(units, return_sequences=True, recurrent_initializer='glorot_uniform')(inputs)
        # out_forward = keras.layers.GlobalAveragePooling1D()(out_forward)
        out_forward = self.attention(out_forward)
        
        out_backward = GRU(units, return_sequences=True, recurrent_initializer='glorot_uniform', go_backwards=True)(inputs)
        # out_backward = keras.layers.GlobalAveragePooling1D()(out_backward)
        out_backward = self.attention(out_backward)
        concate = concatenate([out_backward,out_forward],axis=-1)
        concate = keras.layers.Reshape([1,512])(concate)
        feature = Conv1D(16,3,activation="relu",padding="valid",kernel_initializer="he_uniform",
                         data_format="channels_first")(concate)
        feature = Conv1D(32,3,activation="relu",padding="valid",kernel_initializer="he_uniform",
                         data_format="channels_first")(feature)
        feature = keras.layers.GlobalAveragePooling1D()(feature)
        flat = keras.layers.Flatten()(feature)
        
        
        full_con = Dense(256,activation="relu",kernel_initializer="he_uniform")(flat)
        out = Dense(self.num_actions,kernel_initializer='truncated_normal')(full_con)
        
        return Model(x,out)
    
  
    
   
    
    
    def compile_model(self):
        """
        Choose optimizer and compile model
        """
        if self.optimizer == 'sgd':
            opt = keras.optimizers.SGD(lr=self.learning_rate, momentum=0.9, decay=0.9, nesterov=True)
        elif self.optimizer == 'adam':
            opt = keras.optimizers.Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        elif self.optimizer == 'nadam':
            opt = keras.optimizers.Nadam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
        elif self.optimizer == 'adadelta':
            opt = keras.optimizers.Adadelta(lr=self.learning_rate, rho=0.95, epsilon=1e-08, decay=0.0)
        else:
            opt = keras.optimizers.RMSprop(lr=self.learning_rate, rho=0.9, epsilon=1e-06)

        self.model.compile(optimizer="nadam", loss='mse')
        self.target_model.compile(optimizer="nadam", loss='mse')
        print(self.model.summary())
        
        

    def update_target_network(self):
        """
        Update target DQN
        """
        self.target_model.set_weights(self.model.get_weights())
    
  
          
 
    def train(self, minibatch):
        """
        Train DQN with a mini-batch of samples
        """
        # expand components of minibatch
        prestates, actions, rewards, poststates, terminals = minibatch
       
        prestates=tf.convert_to_tensor(list(prestates))
        poststates=tf.convert_to_tensor(list(poststates))
        
        postq = self.target_model.predict_on_batch(poststates)
        targets = self.model.predict_on_batch(prestates)
        
        maxpostq = postq.max(axis=-1)

        # set_trace()
        # update Q-value targets for actions taken  
        for i, action in enumerate(actions):
            if terminals[i]:  
                targets[i, action] = float(rewards[i])
            else: 
                targets[i, action] = float(rewards[i]) + self.gamma * maxpostq[i]
        
        
        loss=self.model.train_on_batch(prestates,targets)
        return loss
        # set_trace()
    def predict(self, current_state):
        """
        Predict Q-values
        """
        
        current_state=tf.expand_dims(tf.convert_to_tensor(current_state),axis=0)
       
        qvalues = self.model.predict_on_batch(current_state)
        return qvalues


    def save_weights(self, weight_dir):
        """
        Save weights
        """
        self.model.save_weights(weight_dir)
        print('Saved weights to %s ...' % weight_dir)


    def load_weights(self, weight_dir):
        """
        Load weights
        """
        self.model.load_weights(weight_dir)
        print('Loaded weights from %s ...' % weight_dir)


#主函数

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
################################################################################
# Project:  Extracting Action Sequences Based on Deep Reinforcement Learning
# Module:   main
# Author:   Wenfeng Feng 
# Time:     2017.12
################################################################################

import time
# import ipdb
import pickle
import argparse
import tensorflow as tf
import numpy as np
# from EASDRL.utils import get_time, plot_results, str2bool, print_args
# from EASDRL.Agent import Agent
# # from EASDRL.EADQN import DeepQLearner
# from EASDRL.KerasEADQN import DeepQLearner
# from EASDRL.Environment import Environment
# from EASDRL.ReplayMemory import ReplayMemory
from gensim.models import KeyedVectors



def preset_args():
    """
    Preset args
    """
    parser = argparse.ArgumentParser()

    envarg = parser.add_argument_group('Environment')
    envarg.add_argument("--domain",         type=str,       default='wikihow',  help="One of 'win2k', 'cooking' and 'wikihow'")
    envarg.add_argument("--model_dim",      type=str,       default=50,         help="Dimension of the pretrained word vectors (word2vec model)")
    envarg.add_argument("--num_words",      type=int,       default=400,        help="Maximum number of words in a text")
    envarg.add_argument("--word_dim",       type=int,       default=50,         help="Dimension of word embedding")
    envarg.add_argument("--tag_dim",        type=int,       default=50,         help="Dimension of tag embedding")
    envarg.add_argument("--dis_dim",        type=int,       default=50,         help="Dimension of distance embedding")
    envarg.add_argument("--pos_dim",        type=int,       default=50,         help="Dimension of part-of-speech embedding")
    envarg.add_argument("--context_len",    type=int,       default=100,        help="Maximum number of words in the context (of an action name)")
    envarg.add_argument("--reward_assign",  type=list,      default=[1, 2, 3],  help="Reward for essential, optional and exclusive items")
    envarg.add_argument("--reward_base",    type=float,     default=50.0,       help="For reward scaling")
    envarg.add_argument("--object_rate",    type=float,     default=0.07,       help="Percentage of action arguments, a priori")
    envarg.add_argument("--action_rate",    type=float,     default=0.10,       help="Percentage of action names, a priori")
    envarg.add_argument("--use_act_rate",   type=str2bool,  default=True,       help="Whether or not use action_rate, for name extractor")
    envarg.add_argument("--use_act_att",    type=str2bool,  default=False,      help="Whether or not use the given action name to compute attention, for arguments extractor")
    envarg.add_argument("--use_pos",        type=str2bool,  default=True,       help="Whether or not use the part-of-speech of words as input")
    
    memarg = parser.add_argument_group('Replay memory')
    memarg.add_argument("--positive_rate",      type=float,     default=0.9,    help="Percentage of positive samples in a minibatch")
    memarg.add_argument("--priority",           type=str2bool,  default=True,   help="Whether or not apply prioritized sampling")
    memarg.add_argument("--save_replay",        type=str2bool,  default=False,  help="Whether or not save the replay memory")
    memarg.add_argument("--load_replay",        type=str2bool,  default=False,  help="Whether or not load last replay memory")
    memarg.add_argument("--replay_size",        type=int,       default=50000,  help="Size of the replay memory")
    memarg.add_argument("--save_replay_size",   type=int,       default=1000,   help="Size of the replay memory to be saved")
    memarg.add_argument("--save_replay_name",   type=str,       default='data/saved_replay_memory.pkl', help="Directory of saved replay memory")

    netarg = parser.add_argument_group('Deep Q-learning network')
    netarg.add_argument("--batch_size",     type=int,   default=32,     help="Size of minibatch")
    netarg.add_argument("--num_filters",    type=int,   default=32,     help="Number of convolutional filters")
    netarg.add_argument("--dense_dim",      type=int,   default=256,    help="Dimension of the last but two fully-connected layer")
    netarg.add_argument("--num_actions",    type=int,   default=2,      help="Number of actions")
    netarg.add_argument("--optimizer",      type=str,   default='nadam', help="Optimizer")
    netarg.add_argument("--learning_rate",  type=float, default=0.001,  help="Learning rate")
    netarg.add_argument("--dropout",        type=float, default=0.25,   help="Dropout rate")
    netarg.add_argument("--gamma",          type=float, default=0.9,    help="Discount factor")

    antarg = parser.add_argument_group('Agent')
    antarg.add_argument("--exploration_rate_start",     type=float,     default=1,      help="Initial exploration probability for training")
    antarg.add_argument("--exploration_rate_end",       type=float,     default=0.1,    help="Final exploration probability for training")
    antarg.add_argument("--exploration_rate_test",      type=float,     default=0.0,    help="Exploration probability for testing")
    antarg.add_argument("--exploration_decay_steps",    type=int,       default=1000,   help="How many time steps to anneal exploration probability")
    antarg.add_argument("--train_frequency",            type=int,       default=1,      help="Train model after 'train_frequency' time steps")
    antarg.add_argument("--train_repeat",               type=int,       default=1,      help="How many bathes to train at a time step")
    antarg.add_argument("--target_steps",               type=int,       default=5,      help="Copy main network to target network after 'target_steps' time steps")
    antarg.add_argument("--random_play",                type=str2bool,  default=False,  help="Play randomly or not")
    antarg.add_argument("--display_training_result",    type=str2bool,  default=True,   help="Whether or not display the F1 scores after each episode")
    antarg.add_argument("--filter_act_ind",             type=str2bool,  default=True,   help="Whether or not neglact the words of action names when extracting action arguments")

    mainarg = parser.add_argument_group('Main loop')
    mainarg.add_argument("--gui_mode",              type=str2bool,  default=False,  help="Whether or not in the human-agent interaction environment")
    mainarg.add_argument("--gpu_fraction",          type=float,     default=0.2,    help="Fraction of GPU memory to use")
    mainarg.add_argument("--epochs",                type=int,       default=20,     help="Training epochs")
    mainarg.add_argument("--start_epoch",           type=int,       default=0,      help="Training from this epoch")
    mainarg.add_argument("--stop_epoch_gap",        type=int,       default=5,      help="Early stop if no improvement after 'stop_epoch_gap' epochs")
    mainarg.add_argument("--train_episodes",        type=int,       default=50,     help="Test once after training 'train_episodes' texts")
    mainarg.add_argument("--load_weights",          type=str2bool,  default=False,  help="Load weights or not")
    mainarg.add_argument("--save_weights",          type=str2bool,  default=True,   help="Save weights or not")
    mainarg.add_argument("--fold_id",               type=int,       default=0,      help="Which fold of data to be trained")
    mainarg.add_argument("--start_fold",            type=int,       default=0,      help="Start training from this fold of data")
    mainarg.add_argument("--end_fold",              type=int,       default=5,      help="End training after this fold of data")
    mainarg.add_argument("--k_fold",                type=int,       default=5,      help="Number of folds, usually 10 or 5")
    mainarg.add_argument("--result_dir",            type=str,       default='test', help="File name for saving results")
    mainarg.add_argument("--agent_mode",            type=str,       default='act',  help="One of 'act' (name extractor) and 'arg' (arguments extractor)")
    
    return parser.parse_known_args()[0]


def args_init(args):
    """
    args initialization
    """
    with open("/content/drive/My Drive/EASDRL/Glove_emb/{}_word2ids.pkl".format(args.domain),"rb") as f:
        data = pickle.load(f)
    args.word2ids = data
    
    with open("/content/drive/My Drive/EASDRL/data/{}_wordvec_dim50_matrix.pkl".format(args.domain),"rb") as f:
        embedding = pickle.load(f)
    args.emb_matrix = np.array(embedding)
    if args.load_weights:
        args.exploration_rate_start = args.exploration_rate_end
    if args.agent_mode == 'arg':
        args.num_words = 100
        args.train_episodes = 1000
        args.display_training_result = 0
    args.k_fold_indices = '/content/drive/My Drive/EASDRL/data/indices/%s_%s_%d_fold_indices.pkl' % (args.domain, args.agent_mode, args.k_fold)
    args.result_dir = 'results/%s_%s_%s' % (args.domain, args.agent_mode, args.result_dir)
    if args.end_fold > args.k_fold or args.end_fold <= 0:
        args.end_fold = args.k_fold
    return args


def main(args):
    """
    main function, build, train, validate, save and load model
    """
    start = time.time()
    print('Current time is: %s' % get_time())
    print('Starting at main...')
    # store k-fold cross-validation results, including recall, precision, f1 and average reward
    fold_result = {'rec': [], 'pre': [], 'f1': [], 'rw': []}

    # one can continue to train model from the start_fold rather than fold 0 
    for fi in range(args.start_fold, args.end_fold):
        fold_start = time.time()
        args.fold_id = 3
        if args.fold_id == args.start_fold:
            # Initialize environment and replay memory
            env_act = Environment(args, args.agent_mode)
            mem_act = ReplayMemory(args, args.agent_mode)
        else:
            env_act = Environment(args, args.agent_mode)
            mem_act = ReplayMemory(args, args.agent_mode)
            env_act.get_fold_data(3)
            mem_act.reset()


            # ipdb.set_trace()
            # Initialize deep_q_net and agent
        net_act = DeepQLearner(args, args.agent_mode)
        agent = Agent(env_act, mem_act, net_act, args)

            # loop over epochs
        epoch_result = {'rec': [0.0], 'pre': [0.0], 'f1': [0.0], 'rw': [0.0]}
        training_result = {'rec': [], 'pre': [], 'f1': [], 'loss': [], 'rw': []}
        log_epoch = 0
        with open("/content/drive/My Drive/EASDRL/%s_fold%d.txt" % (args.result_dir, args.fold_id), 'w') as outfile:
                # print all args to the screen and outfile
            print_args(args, outfile)

            if args.load_weights:
                print('Loading weights ...')
                filename = '/content/drive/My Drive/EASDRL/model/%s_%s_%d_fold%d.h5' % (args.domain, args.agent_mode, args.k_fold, args.fold_id)
                net_act.load_weights(filename)

            for epoch in range(args.start_epoch, args.start_epoch + args.epochs):
                    # test the model every args.train_episodes or at the end of an epoch
                num_test = -1
                env_act.train_epoch_end_flag = False
                while not env_act.train_epoch_end_flag:
                        # training
                    num_test += 1
                    restart_init = False if num_test > 0 else True
                    tmp_result = agent.train(args.train_steps, args.train_episodes, restart_init)
                    for k in training_result:
                        training_result[k].extend(tmp_result[k])
                        # testing
                    rec, pre, f1, rw = agent.test(args.valid_steps, outfile)

                    if f1 > max(epoch_result['f1']):
                        if args.save_weights:
                            filename = '/content/drive/My Drive/EASDRL/model/%s_%s_%d_fold%d.h5' % (args.domain, args.agent_mode, args.k_fold, args.fold_id)
                            net_act.save_weights(filename)

                        epoch_result['f1'].append(f1)
                        epoch_result['rec'].append(rec)
                        epoch_result['pre'].append(pre)
                        epoch_result['rw'].append(rw)
                        log_epoch = epoch
                        outfile.write('\n\n Best f1 score: {}  best epoch: {}\n'.format(epoch_result, log_epoch))
                        print('\n\n Best f1 score: {}  best epoch: {}\n'.format(epoch_result, log_epoch))
                    else:
                        print("当前最高F1为{}".format(max(epoch_result['f1'])))
                    # if no improvement after args.stop_epoch_gap, break
                if epoch - log_epoch >= args.stop_epoch_gap:
                    outfile.write('\n\nBest f1 score: {}  best epoch: {}\n'.format(epoch_result, log_epoch))
                    print('\nepoch: %d  result_dir: %s' % (epoch, args.result_dir))
                    print('-----Early stopping, no improvement after %d epochs-----\n' % args.stop_epoch_gap)
                    break
            if args.save_replay:
                mem_act.save(args.save_replay_name, args.save_replay_size)
                
                # plot the training process results if you want
                # filename = '%s_fold%d_training_process.pdf'%(args.result_dir, args.fold_id)
                # plot_results(epoch_result, args.domain, filename)
                # outfile.write('\n\n training process:\n{}\n\n'.format(epoch_result))

                # find out the best f1 score in the current fold, add it to fold_result
            best_ind = epoch_result['f1'].index(max(epoch_result['f1']))
            for k in epoch_result:
                fold_result[k].append(epoch_result[k][best_ind])
                outfile.write('{}: {}\n'.format(k, fold_result[k]))
                print(('{}: {}\n'.format(k, fold_result[k])))
                # compute the average f1 and average reward of all fold results up to now
            avg_f1 = sum(fold_result['f1']) / len(fold_result['f1'])
            avg_rw = sum(fold_result['rw']) / len(fold_result['rw'])
            outfile.write('\nAvg f1: {}  Avg reward: {}\n'.format(avg_f1, avg_rw))
            print('\nAvg f1: {}  Avg reward: {}\n'.format(avg_f1, avg_rw))
                
            fold_end = time.time()
            print('Total time cost of fold %d is: %ds' % (args.fold_id, fold_end - fold_start))
            outfile.write('\nTotal time cost of fold %d is: %ds\n' % (args.fold_id, fold_end - fold_start))
        
        tf.compat.v1.reset_default_graph()
    end = time.time()
    print('Total time cost: %ds' % (end - start))
    print('Current time is: %s\n' % get_time())




if __name__ == '__main__':

    args = args_init(preset_args())
    main(args)



Current time is: 2020-11-20_12:30:23
Starting at main...
Initializing the Environment...
len(pos_dict): 36
Getting tenfold indices ...
Loading tenfold indices from /content/drive/My Drive/EASDRL/data/indices/wikihow_act_5_fold_indices.pkl

Spliting data according to indices ...


Get new fold data
training texts: 120	validation texts: 30
self.train_steps: 48000	self.valid_steps: 12000


Initializing ReplayMemory...
Getting tenfold indices ...
Loading tenfold indices from /content/drive/My Drive/EASDRL/data/indices/wikihow_act_5_fold_indices.pkl

Spliting data according to indices ...


Get new fold data
training texts: 120	validation texts: 30
self.train_steps: 48000	self.valid_steps: 12000


Reset the replay memory
Initializing the DQN...
Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)          

In [ ]:
import gensim
import re
model=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/EASDRL/data/wordvec_dim50',binary=True)
import pickle
with open("/content/drive/My Drive/EASDRL/Glove_emb/cooking_word2ids.pkl","rb") as f:
    dic=pickle.load(f)
word_list=list(dic.keys())
word_matrix=[]

for word in word_list:
    word=re.sub(u'\(','',word)
    word=re.sub(u'\)','',word)
    if word not in model.vocab:
        word_matrix.append([0]*50)
    else:
        word_matrix.append(list(model[word]))
f = open("/content/drive/My Drive/EASDRL/data/wordvec_dim50_matrix.pkl","wb")
pickle.dump(word_matrix,f)